In [1]:
import pandas as pd
import numpy as np
from scipy.fft import fft2, fftshift
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [2]:
df = pd.read_csv('/Users/varunchowdary/Desktop/DSLFinalProject/dsl_final_project_ai_vs_real_art/detect ai artwork - resizing images for efficiency/frequency_analysis/image_fft_features.csv')

X = df.drop('Label', axis=1)
y = df['Label']


In [6]:
df

for col in df.columns.tolist():
    print("Column" + col + str(df[col].isna().sum()))
from sklearn.impute import SimpleImputer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)



ColumnMean0
ColumnStd0
ColumnSkewness0
ColumnKurtosis0
ColumnEnergy0
ColumnEntropy74
ColumnLabel0


In [7]:
rf_model = RandomForestClassifier(criterion='entropy', random_state=42)
rf_model.fit(X_train_imputed, y_train)

RandomForestClassifier(criterion='entropy', random_state=42)

Basic Random Forest

In [8]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf_model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8167155425219942
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       328
           1       0.86      0.77      0.81       354

    accuracy                           0.82       682
   macro avg       0.82      0.82      0.82       682
weighted avg       0.82      0.82      0.82       682



Parameter Tuning

In [11]:
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None, 10,],
}


grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy'), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_imputed, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'max_depth': None, 'n_estimators': 300}
Best score: 0.8082850775920082


In [12]:
best_model = grid_search.best_estimator_
y_pred_opt = best_model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred_opt)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred_opt))

Accuracy: 0.8079178885630498
              precision    recall  f1-score   support

           0       0.77      0.86      0.81       328
           1       0.85      0.76      0.80       354

    accuracy                           0.81       682
   macro avg       0.81      0.81      0.81       682
weighted avg       0.81      0.81      0.81       682



Support Vector Machines

This might work because all columns are numerical. It doesnt really work that well.

In [17]:
from sklearn.svm import SVC


Below will be the code for NN

In [13]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [17]:
# Define hyperparameters
input_size = X_train_imputed.shape[1]
hidden_size1 = 100
hidden_size2 = 50
output_size = 2  # Assuming binary classification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create the model
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_imputed, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_imputed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)


In [18]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print("Accuracy:", accuracy)

Accuracy: 0.4809384164222874
